In [ ]:
import os
import pickle
import numpy as np
import yaml
from tqdm import tqdm
from PIL import Image

# Downloading the dataset and cloning the repo

In [ ]:
!git clone https://github.com/brownvc/R3GAN.git
!curl -L https://huggingface.co/brownvc/R3GAN-CIFAR10/resolve/main/network-snapshot-final.pkl --output R3GAN/cifar10.pkl
!curl -L https://raw.githubusercontent.com/NVlabs/stylegan3/refs/heads/main/environment.yml --output R3GAN/environment.yml
!wget -c https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz -P ./datasets/
!mkdir "./datasets/cifar-10"

In [ ]:
import pickle 

with open('R3GAN/cifar10.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [ ]:
!cp /kaggle/input/train-main/train.py /kaggle/working/R3GAN/train.py

In [ ]:
!tar -xzf "./datasets/cifar-10-python.tar.gz"

In [ ]:
def save_images(data, filenames, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for i in tqdm(range(len(filenames))):
        img = data[i]
        img = img.reshape(3, 32, 32).transpose(1, 2, 0)
        img = Image.fromarray(np.uint8(img))
        # print(output_dir, filenames[i].decode('utf-8'))
        img.save(os.path.join(output_dir, filenames[i].decode('utf-8')))


data_batches = ['data_batch_1',
                'data_batch_2',
                'data_batch_3',
                'data_batch_4',
                'data_batch_5']
output_dir = './datasets/cifar-10/'
all_data = []
all_filenames = []

for batch in data_batches:
    with open(os.path.join('./cifar-10-batches-py/', batch), 'rb') as f:
        data = pickle.load(f, encoding='bytes')
    all_data.append(data[b'data'])
    all_filenames.extend(data[b'filenames'])

all_data = np.vstack(all_data)
save_images(all_data, all_filenames, output_dir)

In [ ]:
!zip -r "./datasets/cifar-10.zip" "./datasets/cifar-10/"

# Installing required dependencies

In [ ]:
with open('/kaggle/working/R3GAN/environment.yml') as file_handle:
    environment_data = yaml.safe_load(file_handle)

for dependency in environment_data['dependencies']:
    if isinstance(dependency, dict):
      for lib in dependency['pip']:
        os.system(f'pip install {lib}')

In [ ]:
!pip install Ninja

# Generating images from pretrained models

In [ ]:
!python R3GAN/gen_images.py --seeds=0-63 --outdir=out --class=0 --network="R3GAN/cifar10.pkl"

In [ ]:
!python R3GAN/gen_images.py --seeds=0-7 --outdir=out --network="R3GAN/ffhq-256x256.pkl"

# Training flow

In [ ]:
!python R3GAN/train.py --outdir=./training-runs --data="/kaggle/working/datasets/cifar-10.zip" --gpus=2 --batch=256 --mirror=1 --aug=1 --cond=0 --preset="CIFAR10" --tick=1 --snap=200

In [ ]:
!python R3GAN/train.py --outdir=./training-runs --data=./datasets/ffhq-64x64.zip --gpus=8 --batch=256 --mirror=1 --aug=1 --preset="FFHQ-64" --tick=1 --snap=200

In [ ]:
!python R3GAN/train.py --outdir=./training-runs --data=./datasets/ffhq-256x256.zip --gpus=8 --batch=256 --mirror=1 --aug=1 --preset="FFHQ-256" --tick=1 --snap=200

In [ ]:
!python R3GAN/train.py --outdir=./training-runs --data=./datasets/imagenet-32x32.zip --gpus=32 --batch=4096 --mirror=1 --aug=1 --cond=1 --preset="ImageNet-32" --tick=1 --snap=200

In [ ]:
!python R3GAN/train.py --outdir=./training-runs --data=./datasets/imagenet-64x64.zip --gpus=64 --batch=4096 --mirror=1 --aug=1 --cond=1 --preset="ImageNet-64" --tick=1 --snap=200